[View in Colaboratory](https://colab.research.google.com/github/raviteja8484/ML/blob/master/MSRAVITEJA_BATCH_1_ASSIGNMENT4B.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 32
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 15s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 312s 6ms/step - loss: 1.5153 - acc: 0.4411 - val_loss: 2.7318 - val_acc: 0.3181
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:51 - loss: 1.1673 - acc: 0.5743

50000/50000 [==============================] - 296s 6ms/step - loss: 1.0940 - acc: 0.6046 - val_loss: 1.5499 - val_acc: 0.5366
Epoch 3/50
45568/50000 [==========================>...] - ETA: 24s - loss: 0.9090 - acc: 0.6762

50000/50000 [==============================] - 296s 6ms/step - loss: 0.9029 - acc: 0.6785 - val_loss: 1.0299 - val_acc: 0.6683
Epoch 4/50
50000/50000 [==============================] - 297s 6ms/step - loss: 0.7846 - acc: 0.7208 - val_loss: 1.5438 - val_acc: 0.5744
Epoch 5/50
 3328/50000 [>.............................] - ETA: 4:20 - loss: 0.7444 - acc: 0.7428

50000/50000 [==============================] - 298s 6ms/step - loss: 0.7061 - acc: 0.7535 - val_loss: 0.9274 - val_acc: 0.6995
Epoch 6/50
39680/50000 [======================>.......] - ETA: 57s - loss: 0.6426 - acc: 0.7726

50000/50000 [==============================] - 298s 6ms/step - loss: 0.6408 - acc: 0.7734 - val_loss: 0.7762 - val_acc: 0.7483
Epoch 7/50
50000/50000 [==============================] - 298s 6ms/step - loss: 0.5903 - acc: 0.7947 - val_loss: 1.3157 - val_acc: 0.6537
Epoch 8/50
 1920/50000 [>.............................] - ETA: 4:27 - loss: 0.5425 - acc: 0.8094

...

50000/50000 [==============================] - 297s 6ms/step - loss: 0.1568 - acc: 0.9426 - val_loss: 0.6538 - val_acc: 0.8501
Epoch 45/50
38528/50000 [======================>.......] - ETA: 1:03 - loss: 0.1504 - acc: 0.9452

50000/50000 [==============================] - 297s 6ms/step - loss: 0.1524 - acc: 0.9446 - val_loss: 0.6568 - val_acc: 0.8419
Epoch 46/50
50000/50000 [==============================] - 297s 6ms/step - loss: 0.1543 - acc: 0.9437 - val_loss: 0.6372 - val_acc: 0.8522
Epoch 47/50
 1536/50000 [..............................] - ETA: 4:29 - loss: 0.1196 - acc: 0.9629

50000/50000 [==============================] - 296s 6ms/step - loss: 0.1461 - acc: 0.9473 - val_loss: 0.6715 - val_acc: 0.8452
Epoch 48/50
38528/50000 [======================>.......] - ETA: 1:03 - loss: 0.1401 - acc: 0.9494

50000/50000 [==============================] - 296s 6ms/step - loss: 0.1413 - acc: 0.9488 - val_loss: 0.7013 - val_acc: 0.8440
Epoch 49/50
50000/50000 [==============================] - 297s 6ms/step - loss: 0.1363 - acc: 0.9514 - val_loss: 0.7806 - val_acc: 0.8287
Epoch 50/50
 1536/50000 [..............................] - ETA: 4:28 - loss: 0.1241 - acc: 0.9499

50000/50000 [==============================] - 296s 6ms/step - loss: 0.1365 - acc: 0.9513 - val_loss: 0.5847 - val_acc: 0.8650


In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 26s 3ms/step
Test loss: 0.5846692131325603
Test accuracy: 0.865


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')